<a href="https://colab.research.google.com/github/dasooni/AI_VT2021_LAB1/blob/main/TNM094_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install huggingface_hub
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login  
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
import torch
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling, TextDataset
from sklearn.model_selection import train_test_split

# Model name: AI-Sweden-Models/gpt-sw3-126m, AI-Sweden-Models/gpt-sw3-1.3b
model_name = "AI-Sweden-Models/gpt-sw3-356m"
device = "cuda:0" if torch.cuda.is_available() else "cpu" #in case i run this on my laptop or pc

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [5]:
train_data = TextDataset(tokenizer=tokenizer, file_path="train.csv", block_size=64)
test_data = TextDataset(tokenizer=tokenizer, file_path="test.csv", block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [6]:

training_args = TrainingArguments(output_dir="./results" 
                                ,overwrite_output_dir=True
                                ,num_train_epochs=3
                                , per_device_train_batch_size= 8
                                , per_device_eval_batch_size= 8
                                , save_steps=800
                                , eval_steps=400
                                , warmup_steps=500
                                )

# initialize Trainer
trainer = Trainer(
    model = model,                        
    args = training_args,          
    data_collator = data_collator,
    train_dataset = train_data,
    eval_dataset = test_data
)

trainer.train()
trainer.save_model()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [9]:
from transformers import pipeline

pipe = pipeline("text-generation", model = './results', tokenizer = tokenizer)

[{'generated_text': 'Ge mig ett recept med pannkakor 🍕🍕🍕🍕🍕��'}]

In [10]:
pipe("Ge mig ett recept med pannkakor\n ", max_length=30)

[{'generated_text': 'Ge mig ett recept med pannkakor\n \nPannkakor med sylt och grädde,FOOD\n"Pannkakor med sylt och grädde",'}]